# Testing the Head

**Warning:** Before running this notebook, first make sure you understand the command you run and make sure that the robot can freely move.

**Note:** Also stop all other running Python script or notebook connected to the robot as only one connection can run at the same time.

In [1]:
%matplotlib notebook

import time
import cv2 as cv
import numpy as np

from matplotlib import pyplot as plt

## Connect to the head

In [2]:
from reachy import Reachy, parts

In [3]:
reachy = Reachy(
    #head=parts.Head(io='/dev/ttyUSB*'),
    head=parts.Head(io='ws'),
)

###ReachyPart - init - name:  head
###part_name:  head
###Head.orbita_config:  {'Pc_z': [0, 0, 23], 'Cp_z': [0, 0, 0], 'R': 35.9, 'R0': array([[ 0.49240388, -0.8660254 ,  0.08682409],
       [ 0.85286853,  0.5       ,  0.15038373],
       [-0.17364818,  0.        ,  0.98480775]]), 'hardware_zero': array([115.975,  66.364, 111.107])}
###hardware_zero:  [115.975  66.364 111.107]
###attach_dxl_motors - name:  head


You can now connect your robot in Unity.

## Move the neck

Check that all 3 disks are present and ok.

In [ ]:
for d in reachy.head.neck.disks:
    print(d, d.temperature)

Turn compliant/stiff and check that the head is free or fixed.

In [ ]:
reachy.head.compliant = True

In [ ]:
reachy.head.compliant = False

Go to the base position.

In [ ]:
reachy.head.compliant = False
reachy.head.look_at(1, 0, 0, duration=1, wait=True)

Play some random moves.

In [ ]:
x = 0.5
y = (2 * np.random.rand() - 1) * 0.25
z = (2 * np.random.rand() - 1) * 0.25

duration = 1

reachy.head.look_at(x, y, z, duration=duration, wait=False)

real = []

t0 = time.time()
while time.time() - t0 < duration:
    real.append([d.rot_position for d in reachy.head.neck.disks])
    time.sleep(0.01)
    
plt.figure()
plt.plot(real)

## Move the antennas

Check that we have both antennas.

In [ ]:
reachy.head.motors

Turn them stiff.

In [ ]:
for m in reachy.head.motors:
    m.compliant = False

Make them go to 0

In [ ]:
for m in reachy.head.motors:
    m.goal_position = 0

Make them go to 45

In [ ]:
for m in reachy.head.motors:
    m.goal_position = 45

(check that they both moved)

Make them go to 0 again

In [ ]:
for m in reachy.head.motors:
    m.goal_position = 0

Make them follow a sinus for a few seconds.

In [ ]:
t = np.linspace(0, 10, 1000)
pos = 30 * np.sin(2 * np.pi * 0.5 * t)

for p in pos:
    for m in reachy.head.motors:
        m.goal_position = p
    time.sleep(0.01)

## Access the cameras

*Note: the cameras don't seem to be working in the simulator for reachy v1.2.3. - PC*

Check the right camera.

In [ ]:
success, img = reachy.head.right_camera.read()

if success:
    plt.figure()
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

Check the left camera.

In [ ]:
success, img = reachy.head.left_camera.read()

if success:
    plt.figure()
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))